# Historical Data Simulation

In [35]:
import pandas as pd
from test_api import StreetSmartClient
from get_cars import get_cars
from datetime import datetime
import time

In [89]:
historical = []

num_intersections = 6
num_days = 50

# One record per hour.
records_per_int = num_days * 24
id = 0

for intersection in range(0, num_intersections):
    for i in range(0, records_per_int):
        timestamp = time.time() - (i * 3600)
        hour = datetime.fromtimestamp(timestamp).hour
        cars = get_cars(hour)
        historical.append([
            id,
            intersection,
            timestamp,
            hour,
            cars
        ])

        id = id+1

data = pd.DataFrame(
    historical,
    columns=[
        'id',
        'intersection',
        'timestamp',
        'hour',
        'cars'
    ],
).set_index('id')

In [90]:
data.to_csv('historical.csv')

In [ ]:
!psql $DATABASE_URL -c "\copy traffic_api_historicaldata (id, intersection,timestamp,hour,cars) from 'historical.csv' delimiter ',' csv;"